In [1]:
import numpy as np
from numba import njit
from numba import jit

calculate test distances

In [2]:
    def distance_vectors_periodic(x,l_box):
        # new implementation
        distance_vectors = x[:, None, :] - x[None, :, :]
        np.mod(distance_vectors, l_box, out=distance_vectors)
        mask = distance_vectors > np.divide(l_box, 2.)
        distance_vectors += mask * - np.array(l_box)
        distances = np.linalg.norm(distance_vectors, axis = 2)
        return distances

In [20]:
N_P = 1000
x = np.random.rand(N_P,3)*10
dist=np.asarray(distance_vectors_periodic(x,[10,10,10]))
print(dist)
print(dist[tril_vector])

[[0.         4.88268945 6.20586135 ... 5.75351943 4.93741008 2.68945659]
 [4.88268945 0.         4.76396022 ... 4.8387162  5.76046683 3.77698786]
 [6.20586135 4.76396022 0.         ... 5.49970933 5.66477082 4.91433573]
 ...
 [5.75351943 4.8387162  5.49970933 ... 0.         6.15773546 5.00949706]
 [4.93741008 5.76046683 5.66477082 ... 6.15773546 0.         5.89835888]
 [2.68945659 3.77698786 4.91433573 ... 5.00949706 5.89835888 0.        ]]
[4.88268945 6.20586135 4.76396022 5.52792227 6.6553484  6.58721524]


calculate vectorized hist

In [8]:
bin_res = 400
bin_width = 5 / bin_res
bins = np.arange(bin_res) * bin_width
tril_vector = np.tril_indices(4,-1)
hist = np.zeros(bin_res-1)
def rad_dist_func_not_neighbour(distances):
    dist = distances[tril_vector]    
    hist, trash = np.histogram(dist, bins)
    return hist

In [5]:
%timeit rad_dist_func_not_neighbour(dist)

46.3 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Calculate hist with numba njit, without using a cutoff (except in the numpy hist)

In [5]:
bin_res = 400
bin_width = 5 / bin_res
bins = np.arange(bin_res) * bin_width
N_dist= int(N_P*(N_P-1)/2)
@njit
def rad_dist_func_numba(distances):
    a=0
    _=-1
    dist = np.zeros(N_dist)
    for i in range(len(distances)):
        for j in range(i+1,len(distances)-1):
            _ += 1
            dist[_] =distances[i,j]
    hist, trash = np.histogram(dist, bins)

In [7]:
%timeit rad_dist_func_numba(dist)

18.2 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


calculate hist with numba njit and a cutoff

In [16]:
bin_res = 400
bin_width = 5 / bin_res
bins = np.arange(bin_res) * bin_width
n_dist= int(N_P*(N_P-1)/2)

#@njit  
def hist(dist, bins, r_max):
    hist, trash = np.histogram(rad_dist_func_not_neighbour(distances, bins, r_max), bins)
    return hist

@njit
def rad_dist_func_not_neighbour(distances, bins, r_max):
    """ 
    computes histogramm of distances, without neighbourlists
    """
    dist_index = -1
    dist = []
    for i in range(len(distances)):
        for j in range(i+1,len(distances)):
            actual_dist = distances[i,j]
            if actual_dist <= r_max:
                dist.append(actual_dist)
            
    print(dist)

In [17]:
rad_dist_func_not_neighbour(dist, bins, 5)

[4.030295100377446, 3.540745905837992, 4.959024463256582, 4.638796630686617, 4.346973233184411, 4.999001261589203, 3.192938368632251, 4.050192173231146, 4.284758882969549, 4.520196969204087, 4.395267092275489, 2.462472654271317, 2.8785699113244405, 4.711862329537209, 4.472872477208697, 4.338061544090048, 3.455427603218075, 3.034686562182482, 2.838029057827166, 3.935558430402056, 2.5806195643570504, 3.156467440883234, 4.6417567248059015, 3.2431879353501425, 3.433047289469937, 4.684305355910672, 4.0430844739951945, 4.518972221368906, 4.9731657345511495, 4.612825624188864, 4.416383275123167, 4.81628255426582, 4.11780857933385, 2.2249932144275486, 4.414211444646227, 4.148273961084692, 4.106113727679976, 4.825921390838309, 4.178843022742239, 3.46131273399791, 2.9250891549553413, 4.868275231414255, 2.7757434562661767, 3.645075620740766, 3.4073143142443643, 3.704349639055241, 4.290057183409801, 3.4937608877660713, 4.090083181385808, 2.306237291246462, 4.075814385348877, 4.061890205105682, 3.0

for np histogram, numba njit does´nt make a big difference

In [7]:
a=np.random.rand(1000)
b=np.arange(400) / 400
def hist(a,b):
    a,b=np.histogram(a,b)

def hist_numba(a,b):
    a,b=np.histogram(a,b)

In [ ]:
%timeit hist(a,b)

In [ ]:
%timeit hist_numba(a,b)

In [ ]:
@njit
def test_func():
    a=np.sort(np.random.rand(2))
    return a

In [ ]:
%timeit test_func()

In [ ]:
np.zeros([3,3])

In [58]:
@njit
def rad_dist_func_not_neighbour(distances, bins, r_max):
    """ 
    computes histogramm of distances, without neighbourlists
    """
    dist_0_0 = []
    dist_1_1 = []
    dist_2_2 = []
    dist_0_1 = []
    dist_0_2 = []
    dist_1_2 = []
    for i in range(len(distances)):
        for j in range(i+1,len(distances)):
            actual_dist = distances[i,j]
            if actual_dist <= r_max:
                if p_index[i] == 0 and p_index[j] == 0:
                    dist_0_0.append(actual_dist)
                if p_index[i] == 1 and p_index[j] == 1:
                    dist_1_1.append(actual_dist)
                if p_index[i] == 2 and p_index[j] == 2: 
                    dist_2_2.append(actual_dist)
                if p_index[i] == 0 and p_index[j] == 1 or p_index[i] == 1 and p_index[j]==0:
                    dist_0_1.append(actual_dist)
                if p_index[i] == 0 and p_index[j] == 2 or p_index[i] == 2 and p_index[j]==0:
                    dist_0_2.append(actual_dist)
                if p_index[i] == 2 and p_index[j] == 1 or p_index[i] == 1 and p_index[j]==2:
                    dist_1_2.append(actual_dist)
    return [dist_0_0, dist_1_1, dist_2_2, dist_0_1, dist_0_2, dist_1_2]

@jit
def dist_func(actual_dist,i,j):
    index = p_index[i] * kind_amount + p_index[j]
    dist_[p_index[i] * kind_amount + p_index[j]].append(actual_dist)

In [59]:
p_kinds = [600,400,0]
kind_amount = np.size(p_kinds)
p_index=[]
dist_ = [[-1] for _ in range(kind_amount**2+kind_amount)]
for _ in range(len(p_kinds)):
    p_index = np.append(p_index, [_] * p_kinds[_]).astype(int)

In [60]:
rad_dist_func_not_neighbour(dist, bins, 5)

[[4.8826894523611335,
  4.7378486385723395,
  3.4432608605390453,
  3.341103743162906,
  4.477831002758342,
  2.514172616834964,
  3.1234983256739635,
  2.003503019214149,
  3.408641016769103,
  3.754054133387599,
  4.897369135733238,
  2.0364516738451806,
  3.9880943297907057,
  3.5664564360947746,
  4.635522797167325,
  4.628099047453216,
  3.2668691900239057,
  3.163485134829469,
  4.020268273038163,
  4.935513602013813,
  3.521296110770861,
  3.042487151431066,
  2.608402131232335,
  4.7268594141105575,
  4.30574275611576,
  4.2465035823928465,
  3.654962532387192,
  1.4472878672619134,
  2.1714036296070613,
  4.72426826629883,
  4.367552856957048,
  3.3424425675262177,
  4.13227004812493,
  3.266535284757061,
  4.676895992374911,
  4.2614911125432595,
  4.8794323063285825,
  4.367922003637097,
  4.553471996635572,
  3.047039802801643,
  3.533520554380891,
  3.3479255384166993,
  4.292156870530357,
  4.2359839284734395,
  3.473991465051969,
  4.84790814383242,
  4.735582965931015,
